In [1]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
	[
		layers.RandomFlip("horizontal"),
		layers.RandomRotation(0.1),
		layers.RandomZoom(0.2),
	]
)


inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)

x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32, 64, 128, 256, 512]:
	residual = x
	x = layers.BatchNormalization()(x)
	x = layers.Activation("relu")(x)
	x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)
	x = layers.BatchNormalization()(x)
	x = layers.Activation("relu")(x)
	x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)
	x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
	residual = layers.Conv2D(
	size, 1, strides=2, padding="same", use_bias=False)(residual)
	x = layers.add([x, residual])

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="sigmoid")(x)
x = layers.Dense(40)(x)
outputs = layers.Softmax()(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 180, 180, 3)]        0         []                            
                                                                                                  
 sequential (Sequential)     (None, 180, 180, 3)          0         ['input_1[0][0]']             
                                                                                                  
 rescaling (Rescaling)       (None, 180, 180, 3)          0         ['sequential[0][0]']          
                                                                                                  
 conv2d (Conv2D)             (None, 176, 176, 32)         2400      ['rescaling[0][0]']           
                                                                                              

 conv2d_3 (Conv2D)           (None, 22, 22, 128)          8192      ['add_1[0][0]']               
                                                                                                  
 add_2 (Add)                 (None, 22, 22, 128)          0         ['max_pooling2d_2[0][0]',     
                                                                     'conv2d_3[0][0]']            
                                                                                                  
 batch_normalization_6 (Bat  (None, 22, 22, 128)          512       ['add_2[0][0]']               
 chNormalization)                                                                                 
                                                                                                  
 activation_6 (Activation)   (None, 22, 22, 128)          0         ['batch_normalization_6[0][0]'
                                                                    ]                             
          